In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### the function to get latitude and longitude given an address

In [ ]:
def get_lat_lng_json(address,country='ALL'):
    address = '_'.join(address.split(' '))
    url="https://maps.googleapis.com/maps/api/geocode/json?address=%s" % (address)
    import requests
    response = requests.get(url)
    i=0
    if response.status_code == 200:
        coordinate = []
        if country=='ALL':
            for x in response.json()['results']:
                coordinate.append(x['geometry']['location']['lat'])
                coordinate.append(x['geometry']['location']['lng'])
        else:
            for x in response.json()['results']:
                if x['formatted_address'].split()[-1].lower()==country.lower():
                    i=i+1
                    coordinate.append(x['geometry']['location']['lat'])
                    coordinate.append(x['geometry']['location']['lng'])
                else:
                    continue
            if i==0:
                coordinate.append('Nan')
        return coordinate
    else:
        return None

### the function to extract data from www.realtor.com

In [ ]:
def extract_from_realtor_com(zipcode):
    data = pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude","zipcode"])
    url="http://www.realtor.com/apartments/%s"%(zipcode)
    ori_url="http://www.realtor.com"
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    
    #to get the total_page_count to set the limit of page of the loop 
    try:
        total_page_count = int(page_data_soup.find_all('span',class_="page")[-1].text)
    except:
        total_page_count = 1
        
    #set page_count to go to the next page during the loop
    page_count = 1
    
    #loop through all the pages
    while page_count<=int(total_page_count):
        url="http://www.realtor.com/apartments/%s/pg-%s"%(zipcode,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        #get all the apts on one page
        apt_each_page = page_data_soup.find_all("div",class_="aspect-content")
        #loop through each apt on one page
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="aspect-content")[j]
            #to get the address
            addr = single_result.find('span',class_="listing-street-address").text
            #get the link
            addr_link = ori_url+single_result.find('a').get("href")
            #get the price
            try:      
                ori_price=single_result.find('div',class_="srp-item-price").text.strip()
                price = "".join(ori_price.split(','))
            except:
                price="Nan"
            #get the zipcode to make sure that apt found is indeed the zipcode we need
            try:
                zipcode = page_data_soup.find('span',class_="listing-postal").text
            except:
                zipcode = 'Nan'
            #sometimes one picture is with 2 apartment
            #to get the number of bedroom
            try:
                see_if_mutli=len(single_result.find('li', {"data-label":"property-meta-beds"}).text)
                bed=single_result.find('li', {"data-label":"property-meta-beds"}).text.strip()[0]
            except:
                bed = 'Nan'
            #to get the number of bathroom
            try:
                bath=single_result.find('li', {"data-label":"property-meta-baths"}).text.strip()[0]  
            except:
                bath = 'Nan'
            if see_if_mutli==4:
                coordinate = get_lat_lng_json(addr,country='USA')
                try:
                    lat = get_lat_lng_json(addr,country='USA')[0]
                    lng = get_lat_lng_json(addr,country='USA')[1]
                except:
                    lat = 'Nan'
                    lng = 'Nan'
                data.loc[data.shape[0]]=[addr,price,bed,bath,addr_link,lat,lng,zipcode]
        page_count+=1
    return data

### the function to extract data from www.zillow.com

In [ ]:
def extract_from_zillow_com(zipcode):
    data = pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude","zipcode"])
    url = "http://www.zillow.com/homes/for_rent/Manhattan-New-York-NY-%s"%(zipcode)
    ori_url = "http://www.zillow.com"
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    #to get the total_page_count
    try:
        total_page_count = int(page_data_soup.find("ol",class_="zsg-pagination").text.strip()[-5])
    except:
        total_page_count = 1
    #print(total_page_count)
    page_count = 1
    #loop through all the pages
    while page_count<=int(total_page_count):
        url = "http://www.zillow.com/homes/for_rent/Manhattan-New-York-NY-%s/%s_p/"%(zipcode,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        #get all the apts on one page
        apt_each_page = page_data_soup.find_all("div",class_="zsg-photo-card-content zsg-aspect-ratio-content")
        #loop through each apt on one page
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="zsg-photo-card-content zsg-aspect-ratio-content")[j]
            try:
                addr = single_result.find('span',class_="zsg-photo-card-address").text
                coordinate = get_lat_lng_json(addr,country='USA')
                try:
                    lat = get_lat_lng_json(addr,country='USA')[0]
                    lng = get_lat_lng_json(addr,country='USA')[1]
                except:
                    lat = 'Nan'
                    lng = 'Nan'
                try:
                    addr_link = ori_url+single_result.find('a').get("href")
                except:
                    link = 'Nan'
                try:
                    ori_price = single_result.find('span',class_="zsg-photo-card-price").text
                    price = "".join(ori_price.split(','))
                    try:
                        bed = single_result.find('span',class_="zsg-photo-card-info").text.strip()[0]
                    except:
                        bed ='Nan'
                    try:
                        bath_where = single_result.find('span',class_="zsg-photo-card-info").text.strip()
                        if bath_where[0] == str(1):
                            bath = bath_where[7]
                        else:
                            bath = bath_where[8]
                    except:
                        bath = 'Nan'
                except:
                    for i in single_result.find_all('span',class_="zsg-photo-card-unit"):
                        ori_price = i.text.strip()[1:]
                        price = "".join(ori_price.split(','))
                        bed = i.text.strip()[0]
                        bath = 'Nan'
                        data.loc[data.shape[0]] = [addr,price,bed,bath,addr_link,lat,lng,zipcode]
            except:
                continue
        page_count+=1
    return data

### the function to extract data from www.rent.com

In [ ]:
def extract_from_rent_com(zipcode):
    data=pd.DataFrame(columns = ["address","price/$","bdrms","baths","link","latitude","longitude","zipcode"])
    url="http://www.rent.com/zip-%s"%(zipcode)
    response = requests.get(url)
    page_data_soup = BeautifulSoup(response.content,'html.parser')
    #to get the total listing amount
    total_listings_count=page_data_soup.find('span',class_="total-listings-count").text 
    page_count = 1
    listings_count = 1
    #loop through each page until we go through all the apts
    while listings_count<=int(total_listings_count):
        url = "http://www.rent.com/zip-%s-houses/?page=%s"%(zipcode,page_count)
        response = requests.get(url)
        page_data_soup = BeautifulSoup(response.content,'html.parser')
        ori_url="http://www.rent.com"
        apt_each_page=page_data_soup.find_all('div',class_="prop-info result")
        for j in range(len(apt_each_page)):
            single_result = page_data_soup.find_all('div',class_="prop-info result")[j]
            try:
                add_where = single_result.find('div',class_="prop-title clearfix")
                addr = add_where.find('strong').text
                coordinate = get_lat_lng_json(addr,country='USA')
                try:
                    lat = coordinate[0]
                    lng = coordinate[1]  
                except:
                    lat = 'Nan'
                    lng = 'Nan'
                try:
                    addr_link = ori_url+add_where.find('a').get("href")
                except:
                    addr_link = 'Nan'
                try:
                    price_where = single_result.find('p',class_="prop-rent")       
                    price = price_where.find('span').text.strip()
                except:
                    price = 'Nan'
                try:
                    room_where = single_result.find('div',class_="prop-unit-info bullet-separator")
                except:
                    bed = 'Nan'
                    bath = 'Nan'
                try:
                    bed = room_where.find('span',class_="prop-beds bullet-separator").text.strip()[0]
                except:
                    bed = 'Nan'
                try:
                    bath = room_where.find('span',class_="prop-baths bullet-separator").text.strip()[0]
                except:
                    bath = 'Nan'
                #write the info of each apts into the row of dataframe
                data.loc[data.shape[0]] = [addr,price,bed,bath,addr_link,lat,lng,zipcode]
            except:
                continue
            listings_count+= 1
        #go to next page
        page_count+=1
    return data

### the function to extract data from www.trulia.com

In [ ]:
def extract_from_trulia(zipcode):
    url = 'https://www.trulia.com/for_rent/{}_zip'.format(zipcode)
    response = requests.get(url)
    if response.status_code == 200:
        page = BeautifulSoup(response.content,'html.parser')
        p=[]
    for i in range(100):
        test = page.find('div',{"class":"col cols16 mts txtC srpPagination_list"})
        if test is not None:
            for all in test.find_all('a'):
                p.append(all.text)
            break
        else:
            continue
    try:
        max_page = int(p[-1])   
    except:
        max_page = 1
    l= list()
    for page in range(max_page):
        url = 'https://www.trulia.com/for_rent/{}_zip/{}_p'.format(zipcode,page+1)
        response = requests.get(url)
        if response.status_code == 200:
            page = BeautifulSoup(response.content,'lxml')
            for tag in page.find_all('li',{'class':"hover propertyCard property-data-elem clickable"}):
                #address
                try:
                    address = tag.find('span',{'itemprop':"streetAddress"}).get_text()
                    coordinate = get_lat_lng_json(address,country='USA')
                    #get the latitude and longitude
                    try:
                        lat = get_lat_lng_json(address,country='USA')[0]
                        lng = get_lat_lng_json(address,country='USA')[1]                
                    except:
                        lat = 'Nan'
                        lng = 'Nan'
                    #get the price
                    try:
                        ori_price = tag.find('span',{'class':"typeEmphasize"}).get_text().strip()
                        price = "".join(ori_price.split(','))
                    except:
                        price = 'Nan'
                    #bathroom and bedroom
                    b=[]
                    try:
                        for subtag in tag.find('div',{'class':'col cols3'}).find_all('div'):
                            b.append(subtag.get_text().strip()[0])
                    except:
                        b[0] = 'Nan'
                        b[1] = 'Nan'
                    #link
                    try:
                        link = tag.find('a',{'class':'primaryLink pdpLink activeLink'}).get('href')
                        link = 'https://www.trulia.com' + link
                    except:
                        link = 'Nan'
                    l.append((address,price,b[0],b[1],link,lat,lng,zipcode))
                except:
                    continue
        else:
            print('We cannot find the website')
    data=pd.DataFrame(l,columns=["address","price/$","bdrms","baths","link","latitude","longitude","zipcode"])
    return data



### get the data seperately and combine them together
the reason that we make them seperately is because sometimes if a website fails, we can still get results from other website

In [ ]:
d1_25 = extract_from_realtor_com(10025)
d1_25.to_excel("d1_25.xlsx",sheet_name='sheet 1')

In [ ]:
d2_25 = extract_from_zillow_com(10025)
d2_25.to_excel("d2_25.xlsx",sheet_name='sheet 1')

In [ ]:
d3_25 = extract_from_rent_com(10025)
d3_25.to_excel("d3_25.xlsx",sheet_name='sheet 1')

In [ ]:
d4_25 = extract_from_trulia(10025)
d4_25.to_excel("d4_25.xlsx",sheet_name='sheet 1')

In [ ]:
d1_26 = extract_from_realtor_com(10026)
d1_26.to_excel("d1_26.xlsx",sheet_name='sheet 1')

In [ ]:
d2_26 = extract_from_zillow_com(10026)
d2_26.to_excel("d2_26.xlsx",sheet_name='sheet 1')

In [ ]:
d3_26 = extract_from_rent_com(10026)
d3_26.to_excel("d3_26.xlsx",sheet_name='sheet 1')

In [ ]:
d4_26 = extract_from_trulia(10026)
d4_26.to_excel("d4_26.xlsx",sheet_name='sheet 1')

In [ ]:
d1_27 = extract_from_realtor_com(10027)
d1_27.to_excel("d1_27.xlsx",sheet_name='sheet 1')

In [ ]:
d2_27 = extract_from_zillow_com(10027)
d2_27.to_excel("d2_27.xlsx",sheet_name='sheet 1')

In [ ]:
d3_27 = extract_from_rent_com(10027)
d3_27.to_excel("d3_27.xlsx",sheet_name='sheet 1')

In [ ]:
d4_27 = extract_from_trulia(10027)
d4_27.to_excel("d4_27.xlsx",sheet_name='sheet 1')

In [ ]:
dc_final = pd.concat([d1_25,d2_25,d3_25,d4_25,d1_26,d2_26,d3_26,d4_26,d1_27,d2_27,d3_27,d4_27],ignore_index=True)
dc_final.to_excel('all_data.xlsx',sheet_name='Sheet 1')